In [1]:
import sys    # 시스템
import os     # 시스템

import pandas as pd    # 판다스 : 데이터분석 라이브러리
import numpy as np     # 넘파이 : 숫자, 행렬 데이터 라이브러리

from bs4 import BeautifulSoup     # html 데이터 전처리
import requests
#from selenium import webdriver    # 웹 브라우저 자동화
import time                       # 시간 지연
from tqdm import tqdm_notebook    # 진행상황 표시
#from selenium.webdriver.common.by import By

In [2]:
# CSV 파일 불러오기

keyword = pd.read_csv('C:/Users/socsoft/Downloads/internship_kwj/socsoft/법제처키워드/(v)불합리 규정(108)_키워드.csv', encoding='utf-8')
keyword

,제목
0,“근로자인 조부모도 손자녀 육아휴직 근거 마련해야”
1,노무사 등 손해보험 가입의무 위반기간 따라 과태료 부과
2,‘한의사도 보건소장 우선임용’ 법안 발의 한의협 “조속한 개정 필요”
3,인앱결제 강제 매출액 2% 과징금 구글 애플 `꼼수` 제동
4,인앱결제 강제 때 매출액 2% 과징금 부과 방통위 시행령 마련
...,...
103,"법제처, 지자체 교육청 규칙 중 불합리한 규정 고친다"
104,미추홀구 노력으로 기초지자체 자율권 보장
105,'특허' 일감몰아주기 과세 축소 '없던 일'로(종합)
106,"법제처, 보호소년 수용자 등 기본 인권 확대 행정규칙 7건 정비"


In [3]:
# 데이터 프레임 열에 이름 없을 경우 이름달아 주기

keyword=keyword.rename(columns={'Unnamed: 0':'순번','0':'제목'})

In [4]:
# 데이터 프레임에서 key 값으로 쓸 열을 리스트 형태로 가져오기
 
key = keyword["제목"]

key

0                “근로자인 조부모도 손자녀 육아휴직 근거 마련해야”
1              노무사 등 손해보험 가입의무 위반기간 따라 과태료 부과
2      ‘한의사도 보건소장 우선임용’ 법안 발의 한의협 “조속한 개정 필요”
3            인앱결제 강제 매출액 2% 과징금 구글 애플 `꼼수` 제동
4          인앱결제 강제 때 매출액 2% 과징금 부과 방통위 시행령 마련
                        ...                  
103             법제처, 지자체 교육청 규칙 중 불합리한 규정 고친다
104                    미추홀구 노력으로 기초지자체 자율권 보장
105             '특허' 일감몰아주기 과세 축소 '없던 일'로(종합)
106       법제처, 보호소년 수용자 등 기본 인권 확대 행정규칙 7건 정비
107               특허 거래, 일감몰아주기 과세 제외 '없던 일'로
Name: 제목, Length: 108, dtype: object

In [5]:
# 결과 저장 코드

dict = {}    # 전체 크롤링 데이터를 담을 그릇

inlink_url = []

In [6]:
# 네이버 인링크 추출

# 사이트 주소
for j in key[:5]:
    try:
        url = 'https://search.naver.com/search.naver?where=news&sm=tab_jum&query={}'.format(j)
        headers = {'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36'}
        resp = requests.get(url, headers=headers)
        soup = BeautifulSoup(resp.text)
        #time.sleep(1)
        # 네이버 뉴스 검색 후 제목 가져오기
        n_info = soup.find('a', class_='news_tit')

        # 네이버 인링크 정보
        onlink_info_1 = n_info['onclick'].split('&')[3]
        onlink_info_2 = onlink_info_1.split('=')[1]
        onlink_info = onlink_info_2.split('.')

        # 네이버 인링크 URL 리스트에 저장
        onlink_url = 'https://n.news.naver.com/mnews/article/{}/{}'.format(onlink_info[0], onlink_info[1])
        inlink_url.append(onlink_url)
    except:
        inlink_url.append('None URL')

In [8]:
print(len(inlink_url) ,len(key))

5 108


In [23]:
for i in tqdm_notebook(range(0, len(inlink_url))):
    try:
        for x in range(0, len(inlink_url)):
            
            url2 = inlink_url[x]

            headers = {'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36'}
            resp = requests.get(url2, headers=headers)
            #time.sleep(1)
            #print(resp)
            soup = BeautifulSoup(resp.text)
            
            target_info = {}  # 개별 뉴스 내용을 담을 딕셔너리 생성
            
            # 네이버 인링크 뉴스 정보
            news_info = soup.find('div', class_='newsct')

            # 제목 크롤링
            title = news_info.find('h2', 'media_end_head_headline').text

            # 글쓴이 크롤링
            try:
                nickname = news_info.find('em', class_='media_end_head_journalist_name').text
            except:
                nickname = 'None'
            # 날짜 크롤링
            datetime = news_info.find('span', class_='media_end_head_info_datestamp_time _ARTICLE_DATE_TIME').text

            # 내용 크롤링
            contents = news_info.find('div', class_='newsct_article _article_body')
            content1 = contents.find(class_='go_trans _article_content')
            content = content1.text.replace('\n','')

            # 언론사 이름 크롤링
            company = news_info.find('img')['title']
            
            # 크롤링한 글은 target_info라는 딕셔너리에 담음
            target_info['제목'] = title
            target_info['기자명'] = nickname
            target_info['작성일'] = datetime
            target_info['본문'] = content
            target_info['언론사'] = company
            target_info['url'] = inlink_url[x]

            # 각각의 글은 dict라는 딕셔너리에
            dict[x] = target_info
            # 성공 시 글 제목 출력
            print(x, title)
    except:
        continue
        
print('수집한 글 갯수: ', len(dict))

# 크롤링 결과 데이터 프레임으로 만들기
result_df = pd.DataFrame.from_dict(dict, 'index')

# 데이터 프레임을 엑셀로 저장
result_df.to_excel("naver_news_content.xlsx", encoding='utf-8-sig')

C:\Users\socsoft\AppData\Local\Temp\ipykernel_12868\3142568420.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(0, len(inlink_url))):


  0%|          | 0/5 [00:00<?, ?it/s]

0 “근로자인 조부모도 손자녀 육아휴직 근거 마련해야”
1 노무사 등 손해보험 가입의무 위반기간 따라 과태료 부과
2 ‘한의사도 보건소장 우선임용’ 법안 발의…한의협 “조속한 개정 필요”
3 인앱결제 강제 매출액 2% 과징금… 구글·애플 `꼼수` 제동
4 인앱결제 강제 때 매출액 2% 과징금 부과…방통위 시행령 마련
0 “근로자인 조부모도 손자녀 육아휴직 근거 마련해야”
1 노무사 등 손해보험 가입의무 위반기간 따라 과태료 부과
2 ‘한의사도 보건소장 우선임용’ 법안 발의…한의협 “조속한 개정 필요”
3 인앱결제 강제 매출액 2% 과징금… 구글·애플 `꼼수` 제동
4 인앱결제 강제 때 매출액 2% 과징금 부과…방통위 시행령 마련
0 “근로자인 조부모도 손자녀 육아휴직 근거 마련해야”
1 노무사 등 손해보험 가입의무 위반기간 따라 과태료 부과
2 ‘한의사도 보건소장 우선임용’ 법안 발의…한의협 “조속한 개정 필요”
3 인앱결제 강제 매출액 2% 과징금… 구글·애플 `꼼수` 제동
4 인앱결제 강제 때 매출액 2% 과징금 부과…방통위 시행령 마련
0 “근로자인 조부모도 손자녀 육아휴직 근거 마련해야”
1 노무사 등 손해보험 가입의무 위반기간 따라 과태료 부과
2 ‘한의사도 보건소장 우선임용’ 법안 발의…한의협 “조속한 개정 필요”
3 인앱결제 강제 매출액 2% 과징금… 구글·애플 `꼼수` 제동
4 인앱결제 강제 때 매출액 2% 과징금 부과…방통위 시행령 마련
0 “근로자인 조부모도 손자녀 육아휴직 근거 마련해야”
1 노무사 등 손해보험 가입의무 위반기간 따라 과태료 부과
2 ‘한의사도 보건소장 우선임용’ 법안 발의…한의협 “조속한 개정 필요”
3 인앱결제 강제 매출액 2% 과징금… 구글·애플 `꼼수` 제동
4 인앱결제 강제 때 매출액 2% 과징금 부과…방통위 시행령 마련
수집한 글 갯수:  5


In [22]:
dict

{0: {'제목': '“근로자인 조부모도 손자녀 육아휴직 근거 마련해야”',
  '기자명': '김덕준 기자',
  '작성일': '2021.11.25. 오후 7:04',
  '본문': '법제처, 불합리·불공정 법령 개선 위한국민 아이디어 공모제 결과 발표배우자의 가정폭력이 있으면 어머니가 일시지원복지시설에 거주할 수 있는데 이를 아버지도 일시지원복지시설에 거주할 수 있도록 한 내용이 불합리한 법령 공모전에 최우수상을 받았다. 사진은 최우수상 수상자 곽호청(오른쪽)씨와 이강섭 법제처장. 법제처 제공배우자의 가정폭력이 있으면 어머니가 일시지원복지시설에 거주할 수 있는데 이를 아버지도 일시지원복지시설에 거주할 수 있도록 한 내용이 불합리한 법령 공모전에 최우수상을 받았다. 또 근로자인 조부모도 손자녀를 돌보기 위해 육아휴직을 쓸 수 있도록 관련 법령을 개정해야 한다는 내용도 불합리한 법령에 뽑혔다.법제처는 25일 정부세종청사에서 불합리·불공정 법령을 개선하기 위한 ‘2021년 국민 아이디어 공모제’ 결과를 발표하고 시상식을 개최했다고 밝혔다. 이번 공모에는 총 500건이 접수됐으며 심사위원 평가와 광화문 1번가 국민심사를 거쳐 우수 과제 9건을 선정했다.먼저 최우수상은 가정폭력 피해가정의 어머니뿐만 아니라 아버지도 일시지원복지시설의 지원 대상에 포함하도록 ‘한부모가족지원법’을 개정해야 한다고 제안한 곽호창 씨에게 돌아갔다.현재는 ‘일시지원복지시설’의 지원 대상이 아동의 어머니로 한정돼 있는데, 아동의 아버지를 제외할 합리적인 이유가 없다는 의견이다. 여성가족부는 2022년에 법 개정을 추진할 계획이다.이와 함께 우수상은 장애인 등 이동이 불편한 사람들이 사용하는 화재 피난구조설비의 설치, 유지·관리 기준을 마련해야 한다고 제안한 조봉현 씨에게 돌아갔다.법률에서 5층 이상 공동주택 등의 경우 이동약자가 사용하는 소방설비의 설치, 유지·관리의 세부 내용을 대통령령에 정하도록 하고 있으므로 해당 규정을 보완해야 한다는 취지다. 소방청은 제안의견을 받아들여 2022년에 관련